In [14]:
import numpy as np
import pandas as pd

In [15]:
temp_df = pd.read_csv(r'D:\CODING\PYTHON\NLP\IMDB Dataset_1.csv')

In [16]:
df = temp_df.iloc[:6000]

In [17]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [18]:
df.drop_duplicates(inplace=True)

C:\Users\sinha\AppData\Local\Temp\ipykernel_30628\3006716147.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)


In [19]:
import re
def remove_tags(raw_text):
    cleaned_text = re.sub(re.compile('<.*?>'), '', raw_text)
    return cleaned_text

In [20]:
df['review'] = df['review'].apply(remove_tags)

C:\Users\sinha\AppData\Local\Temp\ipykernel_30628\2336150696.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(remove_tags)


In [21]:
df['review'] = df['review'].apply(lambda x:x.lower())

C:\Users\sinha\AppData\Local\Temp\ipykernel_30628\740760900.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(lambda x:x.lower())


In [22]:
from nltk.corpus import stopwords

sw_list = stopwords.words('english')

df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))

C:\Users\sinha\AppData\Local\Temp\ipykernel_30628\2826946130.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))


In [23]:
df['review']

0       one reviewers mentioned watching 1 oz episode ...
1       wonderful little production. filming technique...
2       thought wonderful way spend time hot summer we...
3       basically there's family little boy (jake) thi...
4       petter mattei's "love time money" visually stu...
                              ...                        
5995    first movie ever saw ashley judd first film vi...
5996    recently saw movie hopes seeing accurate portr...
5997    remember watching movie young, could recall ti...
5998    annie's wig look good. cute pretty enough play...
5999    movies viewed almost 60 years later, yet remai...
Name: review, Length: 5997, dtype: object

In [1]:
# Splitting the dataset into training and testing sets
# 
import gensim

In [1]:
# This function tokenizes the text into sentences 

from nltk import sent_tokenize,word_tokenize
from gensim.utils import simple_preprocess
# import gensim 

In [26]:
# Tokenizing the sentences into words
# This function converts sentences into a list of words
# This is useful for further processing and analysis
# It helps in preparing the text data for machine learning models
# This function is essential for text preprocessing

story = []
for doc in df['review']:
    raw_sent = sent_tokenize(doc)
    for sent in raw_sent:
        story.append(simple_preprocess(sent))
    

In [27]:
# Creating the Word2Vec model
# This model learns word embeddings from the text data
# It captures the semantic relationships between words
# This is useful for various NLP tasks such as text classification, sentiment analysis, and more
# This model is trained on the tokenized sentences
# by default vector size is 100, but we can change it
# This model uses a sliding window of size 5 to learn the context of words

model = gensim.models.Word2Vec(
    window=10,
    min_count=2
)

In [28]:
# Building the vocabulary for the Word2Vec model
# This step is essential for training the model
# It allows the model to learn the relationships between words in the text data
# This is a crucial step in the Word2Vec training process

model.build_vocab(story)

In [29]:
# Training the Word2Vec model
# This step trains the model on the tokenized sentences
# It allows the model to learn the word embeddings based on the context of words in the text data
# The model will learn to predict the context of a word given its surrounding words

model.train(story, total_examples=model.corpus_count, epochs=model.epochs)

(3540362, 3769310)

In [30]:
# Checking the vocabulary size of the Word2Vec model
# This gives us the number of unique words in the model's vocabulary
# This is useful for understanding the model's coverage of the text data
# This can help in evaluating the model's performance and its ability to generalize to unseen data

len(model.wv.index_to_key)

25360

In [31]:
# This function computes the document vector for a given document
# It takes a document as input, removes out-of-vocabulary words, and returns the mean
# of the word vectors for the remaining words
# This is useful for converting text data into numerical representations
# This function is essential for text classification tasks
# It helps in creating a fixed-size vector representation of variable-length documents



def document_vector(doc):
    # remove out-of-vocabulary words
    doc = [word for word in doc.split() if word in model.wv.index_to_key]
    return np.mean(model.wv[doc], axis=0)

In [32]:
# verifying the document vector for the first review
# 100 dimensional vector representation of the first review

document_vector(df['review'].values[0])

array([-0.15418196,  0.5057261 ,  0.23694786,  0.14975455, -0.06969841,
       -0.50280553,  0.09809154,  0.73823863, -0.45555404, -0.23907205,
       -0.17694281, -0.5500562 ,  0.05297895,  0.18089019,  0.08432296,
       -0.43438575, -0.01295695, -0.43371964, -0.07964446, -0.7332231 ,
        0.24429756,  0.15083466,  0.30064094, -0.30927008, -0.13727956,
       -0.06077412, -0.25409055, -0.27741894, -0.31064788, -0.14181776,
        0.4254479 ,  0.02412035,  0.0534478 , -0.3915822 , -0.12990966,
        0.47061905, -0.01361409, -0.219377  , -0.30548164, -0.8787739 ,
       -0.01502748, -0.48016235, -0.03032766,  0.10108441,  0.31130746,
       -0.09495999, -0.45189232, -0.00714784,  0.07962143,  0.23397888,
        0.13204578, -0.26287115, -0.22044665, -0.08311336, -0.3377041 ,
        0.20476297,  0.3633046 ,  0.09289724, -0.4213441 ,  0.10623223,
        0.00498722,  0.1213153 , -0.10336146, -0.10098853, -0.45451337,
        0.31141734,  0.25569165,  0.2290632 , -0.52143466,  0.28

In [33]:
df["review"].values[0]

"one reviewers mentioned watching 1 oz episode hooked. right, exactly happened me.the first thing struck oz brutality unflinching scenes violence, set right word go. trust me, show faint hearted timid. show pulls punches regards drugs, sex violence. hardcore, classic use word.it called oz nickname given oswald maximum security state penitentary. focuses mainly emerald city, experimental section prison cells glass fronts face inwards, privacy high agenda. em city home many..aryans, muslims, gangstas, latinos, christians, italians, irish more....so scuffles, death stares, dodgy dealings shady agreements never far away.i would say main appeal show due fact goes shows dare. forget pretty pictures painted mainstream audiences, forget charm, forget romance...oz mess around. first episode ever saw struck nasty surreal, say ready it, watched more, developed taste oz, got accustomed high levels graphic violence. violence, injustice (crooked guards who'll sold nickel, inmates who'll kill order g

In [34]:
# Importing tqdm for progress bar visualization
# This is useful for tracking the progress of long-running operationsq


from tqdm import tqdm

In [35]:
# Creating document vectors for all reviews
# This step converts each review into a fixed-size vector representation
# It helps in preparing the data for machine learning models
# This function is essential for text classification tasks
# It helps in creating a fixed-size vector representation of variable-length documents
# This is useful for tracking the progress of long-running operations

X = []
for doc in tqdm(df['review'].values):
    X.append(document_vector(doc))

100%|██████████| 5997/5997 [04:49<00:00, 20.70it/s]


In [36]:
# Converting the list of document vectors into a NumPy array
# This is useful for further processing and analysis
# This allows us to work with the document vectors in a structured format

X = np.array(X)

In [37]:
X

array([[-0.15418196,  0.5057261 ,  0.23694786, ..., -0.53482026,
         0.25702262,  0.01263153],
       [-0.12471879,  0.44413525,  0.11436008, ..., -0.6722881 ,
         0.20132318,  0.09869701],
       [-0.12829684,  0.5391613 ,  0.20611449, ..., -0.60511035,
         0.23407303, -0.03036152],
       ...,
       [-0.17182101,  0.49192736,  0.30103138, ..., -0.58288723,
         0.20834984, -0.08847344],
       [-0.16785957,  0.39189914,  0.26536083, ..., -0.5877873 ,
         0.18216936,  0.09898672],
       [-0.08609844,  0.556823  ,  0.30097803, ..., -0.6836062 ,
         0.24791956, -0.05595976]], dtype=float32)

In [38]:
# Verifying the shape of the document vectors
# This gives us the number of documents and the dimensionality of the vectors

X[0]

array([-0.15418196,  0.5057261 ,  0.23694786,  0.14975455, -0.06969841,
       -0.50280553,  0.09809154,  0.73823863, -0.45555404, -0.23907205,
       -0.17694281, -0.5500562 ,  0.05297895,  0.18089019,  0.08432296,
       -0.43438575, -0.01295695, -0.43371964, -0.07964446, -0.7332231 ,
        0.24429756,  0.15083466,  0.30064094, -0.30927008, -0.13727956,
       -0.06077412, -0.25409055, -0.27741894, -0.31064788, -0.14181776,
        0.4254479 ,  0.02412035,  0.0534478 , -0.3915822 , -0.12990966,
        0.47061905, -0.01361409, -0.219377  , -0.30548164, -0.8787739 ,
       -0.01502748, -0.48016235, -0.03032766,  0.10108441,  0.31130746,
       -0.09495999, -0.45189232, -0.00714784,  0.07962143,  0.23397888,
        0.13204578, -0.26287115, -0.22044665, -0.08311336, -0.3377041 ,
        0.20476297,  0.3633046 ,  0.09289724, -0.4213441 ,  0.10623223,
        0.00498722,  0.1213153 , -0.10336146, -0.10098853, -0.45451337,
        0.31141734,  0.25569165,  0.2290632 , -0.52143466,  0.28

In [39]:
# Importing LabelEncoder for encoding the sentiment labels
# This is useful for converting categorical labels into numerical format
# This is essential for training machine learning models

from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

y = encoder.fit_transform(df['sentiment'])

In [40]:
y

array([1, 1, 1, ..., 0, 0, 1])

In [41]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [42]:
# Importing RandomForestClassifier for training the model
# This is a popular machine learning algorithm for classification tasks
# It is an ensemble method that combines multiple decision trees to improve accuracy
# This model is useful for text classification tasks

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [43]:
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.7183333333333334